In [1]:
import os
import subprocess
import sys
import shutil

In [2]:
BASE_DIR = "/Users/aia04/Desktop/metal_gpt-oss"
REPO_NAME = "gpt-oss"
PROJECT_DIR = os.path.join(BASE_DIR, REPO_NAME)
MODEL_NAME = "gpt-oss-20b"
REPO_URL = "https://github.com/openai/gpt-oss.git"
REQUIRED_PYTHON_VERSION = (3, 12)

In [3]:
def run_command(command, working_dir=None, env=None, shell=False):
    """Executa um comando no terminal e imprime a saída em tempo real."""
    print(f"\n[EXECUTANDO]: {' '.join(command if isinstance(command, list) else [command])}")
    try:
        process = subprocess.Popen(
            command,
            cwd=working_dir,
            env=env,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT, # Redireciona o erro para a saída padrão
            text=True,
            shell=shell,
            bufsize=1 # Line-buffered
        )

        for line in iter(process.stdout.readline, ''):
            print(line, end='')

        process.wait()
        if process.returncode != 0:
            raise subprocess.CalledProcessError(process.returncode, command)
        
        print(f"\n[SUCESSO]: Comando concluído com êxito.")
        return True

    except Exception as e:
        print(f"\n[ERRO] ao executar o comando: {e}")
        return False

In [5]:
def main():
    """
    Script principal para configurar e executar o gpt-oss com Metal.
    """
    # --- Passo 0: Verificar a Versão do Python ---
    print("--- Passo 0: A verificar a versão do Python ---")
    current_version = sys.version_info
    if not (current_version.major == REQUIRED_PYTHON_VERSION[0] and current_version.minor == REQUIRED_PYTHON_VERSION[1]):
        print(f"[ERRO] Versão do Python incorreta!")
        print(f"Você está a usar: {current_version.major}.{current_version.minor}.{current_version.micro}")
        print(f"Este script requer Python {REQUIRED_PYTHON_VERSION[0]}.{REQUIRED_PYTHON_VERSION[1]}.")
        print("Por favor, use o pyenv ou outro método para mudar para a versão correta e execute novamente.")
        sys.exit(1)
    print(f"Versão do Python ({current_version.major}.{current_version.minor}) correta.")

    # --- Passo 1: Clonar o Repositório ---
    print("\n--- Passo 1: A clonar o repositório ---")
    if not os.path.exists(BASE_DIR):
        os.makedirs(BASE_DIR)

    if not os.path.exists(PROJECT_DIR):
        if not run_command(["git", "clone", REPO_URL], working_dir=BASE_DIR):
            sys.exit("Falha ao clonar o repositório. A abortar.")
    else:
        print(f"Repositório já existe em '{PROJECT_DIR}'. A saltar a clonagem.")

    # --- Passo 2: Criar e Ativar Ambiente Virtual ---
    print("\n--- Passo 2: A configurar o ambiente virtual ---")
    venv_path = os.path.join(PROJECT_DIR, "venv")
    if not os.path.exists(venv_path):
        print("A criar o ambiente virtual com Python 3.12...")
        if not run_command([sys.executable, "-m", "venv", "venv"], working_dir=PROJECT_DIR):
            sys.exit("Falha ao criar o ambiente virtual. A abortar.")
    else:
        print("Ambiente virtual já existe.")

    # Definir os caminhos para os executáveis do venv
    pip_executable = os.path.join(venv_path, "bin", "pip")
    python_executable = os.path.join(venv_path, "bin", "python")
    hf_executable = os.path.join(venv_path, "bin", "huggingface-cli")

    # --- Passo 3: Instalar Dependências com Suporte a Metal ---
    print("\n--- Passo 3: A instalar dependências (pode demorar vários minutos) ---")
    install_env = os.environ.copy()
    install_env["GPTOSS_BUILD_METAL"] = "1"
    install_command = f'"{pip_executable}" install -e ".[metal]"'
    if not run_command(install_command, working_dir=PROJECT_DIR, env=install_env, shell=True):
        print("\n[DICA] Se a instalação falhar, verifique se tem o Xcode Command Line Tools, cmake e ninja instalados.")
        print("Execute: xcode-select --install")
        print("E: brew install cmake ninja")
        sys.exit("Falha ao instalar as dependências. A abortar.")
    
    # --- Passo 4: Baixar os Pesos do Modelo e Corrigir o Caminho ---
    print("\n--- Passo 4: A baixar os pesos do modelo ---")
    model_dir = os.path.join(PROJECT_DIR, MODEL_NAME, "metal")
    model_bin_path = os.path.join(model_dir, "model.bin")
    
    if not os.path.exists(model_bin_path):
        if not run_command([hf_executable, "download", f"openai/{MODEL_NAME}", "--include", "metal/*", "--local-dir", model_dir], working_dir=PROJECT_DIR):
            sys.exit("Falha ao baixar os pesos do modelo. A abortar.")
        
        # Corrigir o caminho se for criado um subdiretório 'metal' extra
        extra_metal_dir_model_path = os.path.join(model_dir, "metal", "model.bin")
        if os.path.exists(extra_metal_dir_model_path):
            print("A corrigir o caminho do ficheiro do modelo...")
            shutil.move(extra_metal_dir_model_path, model_bin_path)
            shutil.rmtree(os.path.join(model_dir, "metal")) # Remover a pasta extra vazia
    else:
        print("Ficheiro do modelo já existe.")

    # --- Passo 5: Executar a Inferência ---
    print("\n--- Passo 5: A executar a geração de texto ---")
    prompt = "why did the chicken cross the road?"
    generate_script = os.path.join(PROJECT_DIR, "gpt_oss/metal/examples/generate.py")
    
    if not run_command([python_executable, generate_script, model_bin_path, "-p", prompt], working_dir=PROJECT_DIR):
        sys.exit("Falha ao executar o script de geração.")

    print("\n\n🎉 --- Processo concluído com sucesso! --- 🎉")

In [6]:
if __name__ == "__main__":
    main()

--- Passo 0: A verificar a versão do Python ---
Versão do Python (3.12) correta.

--- Passo 1: A clonar o repositório ---
Repositório já existe em '/Users/aia04/Desktop/metal_gpt-oss/gpt-oss'. A saltar a clonagem.

--- Passo 2: A configurar o ambiente virtual ---
Ambiente virtual já existe.

--- Passo 3: A instalar dependências (pode demorar vários minutos) ---

[EXECUTANDO]: "/Users/aia04/Desktop/metal_gpt-oss/gpt-oss/venv/bin/pip" install -e ".[metal]"
Obtaining file:///Users/aia04/Desktop/metal_gpt-oss/gpt-oss
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing